## Voice Authentication and Face Recognition

In [21]:
import numpy as np
import os
import glob
import pickle
import time
from numpy import genfromtxt

np.set_printoptions()

import pyaudio
from IPython.display import Audio, display, clear_output
import wave
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture 
import warnings
warnings.filterwarnings("ignore")

from sklearn import preprocessing
import python_speech_features as mfcc

## Audio processing

In [22]:
#Calculate and returns the delta of given feature vector matrix
def calculate_delta(array):
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first = 0
            else:
                first = i-j
            if i+j > rows -1:
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

#convert audio to mfcc features
def extract_features(audio,rate):    
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True, nfft=1103)
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)

    #combining both mfcc features and delta
    combined = np.hstack((mfcc_feat,delta)) 
    return combined

## Add a New User

In [31]:
def add_user():
    name = input("Enter Name:")
    #Voice authentication
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    CHUNK = 1024
    RECORD_SECONDS = 10
    
    source = "./voice_database/" + name
    
    try:
        os.mkdir(source)


        for i in range(3):
            audio = pyaudio.PyAudio()

            if i == 0:
                j = 3
                while j>=0:
                    time.sleep(1.0)
                    print("Speak your name in {} seconds".format(j))
                    clear_output(wait=True)

                    j-=1

            elif i ==1:
                print("Say the pass phrase...")
                time.sleep(0.5)

            else:
                print("Say the pass phrase...")
                time.sleep(0.5)

            # start Recording
            stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

            print("recording...")
            frames = []

            for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
                data = stream.read(CHUNK)
                frames.append(data)

            # stop Recording
            stream.stop_stream()
            stream.close()
            audio.terminate()

            # saving wav file of speaker
            waveFile = wave.open(source + '/' + str((i+1)) + '.wav', 'wb')
            waveFile.setnchannels(CHANNELS)
            waveFile.setsampwidth(audio.get_sample_size(FORMAT))
            waveFile.setframerate(RATE)
            waveFile.writeframes(b''.join(frames))
            waveFile.close()
            print("Done")

        dest =  "./gmm_models/"
        count = 1

        for path in os.listdir(source):
            path = os.path.join(source, path)

            features = np.array([])

            # reading audio files of speaker
            (sr, audio) = read(path)

            # extract 40 dimensional MFCC & delta MFCC features
            vector   = extract_features(audio,sr)

            if features.size == 0:
                features = vector
            else:
                features = np.vstack((features, vector))

            # when features of 3 files of speaker are concatenated, then do model training
            if count == 3:    
                gmm = GaussianMixture(n_components = 16, max_iter = 200, covariance_type='diag',n_init = 3)
                gmm.fit(features)

                # saving the trained gaussian model
                pickle.dump(gmm, open(dest + name + '.gmm', 'wb'))
                print(name + ' added successfully') 

                features = np.asarray(())
                count = 0
            count = count + 1
            
    except:
        print('Username already exist, try another user name.')

if __name__ == '__main__':
    add_user()

Enter Name:mohnish
Username already exist, try another user name.


## Voice Authentication and Face Recognition

In [46]:
def recognize():
    # Voice Authentication
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    CHUNK = 1024
    RECORD_SECONDS = 10
    FILENAME = "./test.wav"

    audio = pyaudio.PyAudio()
   
    # start Recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

    print("recording...")
    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("finished recording")


    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # saving wav file 
    waveFile = wave.open(FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

    modelpath = "./gmm_models/"

    gmm_files = [os.path.join(modelpath,fname) for fname in 
                os.listdir(modelpath) if fname.endswith('.gmm')]

    models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]

    speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
                in gmm_files]
  
    if len(models) == 0:
        print("No Users in the Database!")
        return
        
    #read test file
    sr,audio = read(FILENAME)
    
    # extract mfcc features
    vector = extract_features(audio,sr)
    log_likelihood = np.zeros(len(models)) 
    diff=np.zeros(len(models))
    #checking with each model one by one
    for i in range(len(models)):
        gmm = models[i]         
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
        print("Score for ",speakers[i]," is:",log_likelihood[i])

    pred = np.argmax(log_likelihood)
    identity = speakers[pred]
    print(log_likelihood[pred])
    count=0
    for i in range(len(models)):
        diff[i]=log_likelihood[pred]-log_likelihood[i]
        if diff[i]>0.75:
            count=count+1
    print(diff)
    print(count)
    # if voice not recognized than terminate the process
    if identity == 'unknown' or identity == 'unknown2' or identity == 'unknown3' or identity == 'unknown4' or identity == 'unknown5':
            print("Not Recognized! Try again...")
            return
    else:
        if count==len(models)-1:
            print( "Recognized as - ", identity)
        else:
            print( "Not recognised try again")
        
if __name__ == '__main__':
    recognize()

recording...
finished recording
Score for  girish  is: -22.89739365973171
Score for  janhavi  is: -23.063866813521113
Score for  mahima  is: -24.069592228347823
Score for  mahimaB  is: -23.353151687374208
Score for  mohnish  is: -23.21115665747401
Score for  ovesh  is: -24.04333524076435
Score for  public  is: -29.215341298766766
Score for  purnima  is: -23.340144976407785
Score for  shloki  is: -26.022879861114216
Score for  supriya  is: -23.387671100914115
Score for  unknown  is: -33.69290264613979
Score for  unknown2  is: -28.319961181972683
Score for  unknown3  is: -32.5875711578589
Score for  unknown4  is: -24.375220148087074
Score for  unknown5  is: -26.75448975135713
Score for  vinaya  is: -23.43030783239262
-22.89739365973171
[ 0.          0.16647315  1.17219857  0.45575803  0.313763    1.14594158
  6.31794764  0.44275132  3.1254862   0.49027744 10.79550899  5.42256752
  9.6901775   1.47782649  3.85709609  0.53291417]
10
Not recognised try again


## Delete User

In [32]:
# deletes a registered user from database
def delete_user():
    try:
        name = input("Enter name of the user:")
        # remove the speaker wav files and gmm model
        [os.remove(path) for path in glob.glob('./voice_database/' + name + '/*')]
        os.removedirs('./voice_database/' + name)
        os.remove('./gmm_models/' + name + '.gmm')
    except:
        print('No such user in database.')

delete_user()

Enter name of the user:mkn
No such user in database.
